In [1]:
import os
import re
import cv2
import random
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNEL = 3
NUM_CLASSES = 32
BATCH_SIZE = 32
DROPOUT = 0.3

IMG_PATH = "./data/CamVid/train/"
MASK_PATH = "./data/CamVid/train_labels/"


seed = 42
random.seed = seed
np.random.seed = seed

In [2]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
filename =os.listdir(IMG_PATH)

In [38]:
data = np.array(data)

In [39]:
data.shape

(369, 2, 224, 224, 3)

In [52]:
mask = data[:,1,:]

In [54]:
y_clas = torch.tensor([ColorsToClass(seg) for seg in mask])

In [61]:
y_clas[1].view(1,32)

torch.Size([1, 32])

In [9]:
from dataset import Camvid

In [10]:
camvid = Camvid()

In [11]:
import os

from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.autograd import Variable
from torch.utils.data import DataLoader

from tqdm import tqdm
import click
import numpy as np

from pspnet import PSPNet
from dataset import Camvid


models = {
    'squeezenet': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size =512, deep_features_size=256, backend='squeezenet'),
    'densenet': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=1024, deep_features_size=512, backend='densenet'),
    'resnet18': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=512, deep_features_size=256, backend='resnet18'),
    'resnet34': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=512, deep_features_size=256, backend='resnet34'),
    'resnet50': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet50'),
    'resnet101': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet101'),
    'resnet152': lambda: PSPNet(sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet152')
}


def build_network(snapshot, backend):
    epoch = 0
    backend = backend.lower()
    net = models[backend]()
    net = nn.DataParallel(net)
    if snapshot is not None:
        _, epoch = os.path.basename(snapshot).split('_')
        epoch = int(epoch)
        net.load_state_dict(torch.load(snapshot))
        logging.info("Snapshot for epoch {} loaded from {}".format(epoch, snapshot))
    net.double()
    net = net.cuda()
    return net, epoch


def train(backend = 'resnet18', snapshot = None, crop_x = 256, crop_y = 256, batch_size=4,\
          alpha = 1.0, epochs=30, start_lr=0.001, milestones = '10,20,30', gpu = '0'):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu
    net, starting_epoch = build_network(snapshot, backend)
   
   
    # models_path = os.path.abspath(os.path.expanduser(models_path))
    # os.makedirs(models_path, exist_ok=True)
    print('--------------------')


    '''
        To follow this training routine you need a DataLoader that yields the tuples of the following format:
        (Bx3xHxW FloatTensor x, BxHxW LongTensor y, BxN LongTensor y_cls) where
        x - batch of input images,
        y - batch of groung truth seg maps,
        y_cls - batch of 1D tensors of dimensionality N: N total number of classes, 
        y_cls[i, T] = 1 if class T is present in image i, 0 otherwise
    '''
    
    
    train_loader, class_weights, n_images = None, None, None



    print(f'Loading data......')

#     camvid = Camvid()
    loader = DataLoader(camvid, batch_size=4, num_workers= 0)
    print(f'Loaded')
    optimizer = optim.Adam(net.parameters(), lr=start_lr)
    scheduler = MultiStepLR(optimizer, milestones=[int(x) for x in milestones.split(',')])
    
    for epoch in range(starting_epoch, starting_epoch + epochs):
        seg_criterion = nn.NLLLoss2d()
        cls_criterion = nn.BCEWithLogitsLoss()
        epoch_losses = []
        train_iterator = tqdm(loader, total= len(camvid) // batch_size + 1)
        net.train()
        for x, y, y_cls in train_iterator:
            # steps += batch_size
            optimizer.zero_grad()
            x, y, y_cls = Variable(x).cuda(), Variable(y).cuda(), Variable(y_cls).cuda()
            x = x.permute(0,3,1,2)
            print(x.shape)

            out, out_cls = net(x)
            seg_loss, cls_loss = seg_criterion(out, y), cls_criterion(out_cls, y_cls)
            loss = seg_loss + alpha * cls_loss
            epoch_losses.append(loss.data[0])
            status = '[{0}] loss = {1:0.5f} avg = {2:0.5f}, LR = {5:0.7f}'.format(
                epoch + 1, loss.data[0], np.mean(epoch_losses), scheduler.get_lr()[0])
            print(status)
            train_iterator.set_description(status)
            loss.backward()
            optimizer.step()
        scheduler.step()
        torch.save(net.state_dict(), os.path.join(models_path, '_'.join(["PSPNet", str(epoch + 1)])))
        train_loss = np.mean(epoch_losses)

In [12]:
train()

  0%|                                                                                           | 0/93 [00:00<?, ?it/s]

--------------------
Loading data......
Loaded
torch.Size([4, 3, 224, 224])


  0%|                                                                                           | 0/93 [00:00<?, ?it/s]


NameError: name 'residula' is not defined